##Código da rede neural

In [ ]:
import numpy as np
import math

ETA = 0.5
MOMENTUM = 0.5

def sigmoid(x):
  s = 1/(1 + math.exp(-x))
  return s

class Net:
  def __init__(self, layer_sizes):
    if len(layer_sizes) < 3:
      raise ValueError('len(layer_sizes) < 3. Must have at least 3 layers')
    
    self.layer_sizes = layer_sizes
    self.layers = []
  
    for i in range(1, len(layer_sizes)):
      self.layers.append( Layer(layer_sizes[i], layer_sizes[i -1]) )

  
  def forward(self, inputs):
    out = self.layers[0].forward(inputs)
    for i in range(1, len(self.layers)):
      out = self.layers[i].forward(out)
    return out

  def backpropagation(self, expected_outputs):
    #realiza um out_backpropagation no último layer
    forward_layer_copy = np.copy(self.layers[-1].weight_matrix)
    self.layers[-1].out_backpropagation(expected_outputs)

    #percorre do penúltimo layer até o primeiro layer fazendo hidden_backpropagation
    for i in range(len(self.layers) - 2, -1, -1):
      aux = np.copy(self.layers[i].weight_matrix)
      self.layers[i].hidden_backpropagation(forward_layer_copy, self.layers[i+1].error)
      forward_layer_copy = aux

  def online_train(self, input_batch, output_batch, cicles):
    for i in range(cicles):
      for input_line, output_line in zip(input_batch, output_batch):
        self.forward(input_line.reshape(-1, 1))
        self.backpropagation(output_line.reshape(-1, 1))   

class Layer:
  def __init__(self, num_cells, num_inputs):
    self.num_inputs = num_inputs
    self.num_cells = num_cells
    self.weight_matrix = np.random.uniform(low=-1, high=1, size=(num_cells, num_inputs))
    self.momentum_matrix = np.zeros(shape = (num_cells, num_inputs))
    self.bias = np.random.uniform(low =-1, high= 1, size=(num_cells, 1))
    self.momentum_bias = np.zeros(shape = (num_cells, 1))

  def forward(self, inputs):
    result = self.weight_matrix.dot(inputs)
    result += self.bias
    self.last_outputs = np.vectorize(sigmoid)(result)
    self.last_inputs = inputs 
    return self.last_outputs
  
  def out_backpropagation(self, expected_outputs):
    error = np.vectorize(lambda t,s : (t - s)*s *(1 - s) )(expected_outputs, self.last_outputs)
    delta_matrix = ETA*error.dot(self.last_inputs.T) + self.momentum_matrix
    delta_bias = ETA*error + self.momentum_bias
    self.weight_matrix += delta_matrix 
    self.bias += delta_bias
    self.error = error
    self.momentum_matrix = delta_matrix * MOMENTUM
    self.momentum_bias = delta_bias*MOMENTUM

  def hidden_backpropagation(self, forward_layer_weights, forward_layer_errors):
    sum_errors = forward_layer_weights.T.dot(forward_layer_errors)
    error = np.vectorize(lambda t,s : t*s *(1 - s) )(sum_errors, self.last_outputs)
    delta_matrix = ETA*error.dot(self.last_inputs.T) + self.momentum_matrix
    delta_bias = ETA*error + self.momentum_bias
    self.weight_matrix += delta_matrix
    self.bias += delta_bias
    self.error = error
    self.momentum_matrix = delta_matrix * MOMENTUM
    self.momentum_bias = delta_bias*MOMENTUM

def normalize_inputs(inputs):
  t = inputs.T
  for idx in range(len(t)):
    row = t[idx]
    u = np.mean(row)
    s = np.std(row)
    t[idx] = np.array(list(map(lambda x: (x - u)/s, row)))
  return t.T


#Base de dados de vinho

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

TEST_PROPORTION = 0.2
CICLES = 10
ETA = 0.1
MOMENTUM = 0.1

df = pd.read_csv('/content/wine.data')


#transformando as classes em 3 colunas de booleans:
class_columns = pd.DataFrame(0, range(df.shape[0]), columns=range(3))

for i in range(df.shape[0]):
  classe = df.iloc[i, 0] 
  if classe == 1:
    class_columns.iloc[i, 0] = 1
  elif classe == 2:
    class_columns.iloc[i, 1] = 1
  elif classe == 3:
    class_columns.iloc[i, 2] = 1

df = pd.concat([class_columns, df], axis=1)


def wine_accuracy(test_proportion, cicles, eta, momentum, hidden_layers_num):
  TEST_PROPORTION = test_proportion
  CICLES = cicles
  ETA = eta
  MOMENTUM = momentum
  # embaralha as instâncias 
  shuffled = df.sample(frac=1)

  #numero de instancias do teste
  test_size = int(round(df.shape[0]*TEST_PROPORTION))

  #particiona em grupo de teste e grupo de treino
  teste = shuffled.iloc[0: test_size]
  treino = shuffled.iloc[test_size:]

  net_struct = []
  if hidden_layers_num == 2:
    net_struct = [13, 8, 8, 3]
  else:
    net_struct = [13, 8, 3]
  net = Net(net_struct)

  #TREINO

  #inputs contem os 13 atributos do vinho
  inputs = treino.drop(df.columns[[0, 1, 2, 3]], axis=1).to_numpy()
  #targets contem a classe do vinho
  targets = treino.iloc[:, [0, 1, 2, 3]].to_numpy()

  inputs = normalize_inputs(inputs)
  net.online_train(inputs, targets[:, :-1], CICLES)

  #TESTE
  inputs = teste.drop(df.columns[[0, 1, 2, 3]], axis=1).to_numpy()
  targets = teste.iloc[:, [0, 1, 2, 3]].to_numpy()
  inputs = normalize_inputs(inputs)

  errors = 0
  for input_line, output_line in zip(inputs, targets[:,-1]):
    res = net.forward(input_line.reshape(-1, 1))
    predicted_class = np.argmax(res.flatten()) + 1
    if predicted_class != output_line:
      errors += 1

  acc = 1.0 - (errors/teste.shape[0])
  return acc


0.9714285714285714

In [ ]:
hidden_layer_num = [1,2]
num_cicles = [10,20,50,100]
eta_arr = [0.1,0.3,0.5,0.7]
momentum_arr = [0.3,0.4,0.5,0.6]
test_prop = [0.2,0.4,0.6,0.8]

#testes de acurácia:

#hidden_layers:
print("Número de Camadas: ")
for i in range(len(hidden_layer_num)):
  aux = []
  for j in range(5):
    aux.append(wine_accuracy(0.2, 20, 0.1, 0.1, hidden_layer_num[i]))
  print(f"Valor {hidden_layer_num[i]} : {np.mean(aux)}")

#num_cicles:
print("Número de Ciclos: ")
for i in range(len(num_cicles)):
  aux = []
  for j in range(5):
    aux.append(wine_accuracy(0.2, num_cicles[i], 0.1, 0.1, 1))
  print(f"Valor {num_cicles[i]} : {np.mean(aux)}")

#eta_arr:
print("Velocidade de aprendizado: ")
for i in range(len(eta_arr)):
  aux = []
  for j in range(5):
    aux.append(wine_accuracy(0.2, 20, eta_arr[i], 0.1, 1))
  print(f"Valor {eta_arr[i]} : {np.mean(aux)}")

#momentum_arr:
print("Variação do Momentum: ")
for i in range(len(momentum_arr)):
  aux = []
  for j in range(5):
    aux.append(wine_accuracy(0.2, 20, 0.1, momentum_arr[i], 1))
  print(f"Valor {momentum_arr[i]} : {np.mean(aux)}")

#test_prop:
print("Proporção de teste: ")
for i in range(len(test_prop)):
  aux = []
  for j in range(5):
    aux.append(wine_accuracy(test_prop[i], 20, 0.1, 0.1, 1))
  print(f"Valor {test_prop[i]} : {np.mean(aux)}")

Número de Camadas: 
Valor 1 : 0.96
Valor 2 : 0.9542857142857144
Número de Ciclos: 
Valor 10 : 0.9657142857142859
Valor 20 : 0.9771428571428572
Valor 50 : 0.9542857142857143
Valor 100 : 1.0
Velocidade de aprendizado: 
Valor 0.1 : 0.9428571428571428
Valor 0.3 : 0.9828571428571429
Valor 0.5 : 0.9657142857142856
Valor 0.7 : 0.9714285714285715
Variação do Momentum: 
Valor 0.3 : 0.9771428571428572
Valor 0.4 : 0.9485714285714286
Valor 0.5 : 0.96
Valor 0.6 : 0.9485714285714286
Proporção de teste: 
Valor 0.2 : 0.9771428571428572
Valor 0.4 : 0.9830985915492958
Valor 0.6 : 0.9716981132075471
Valor 0.8 : 0.9267605633802816


#Base de dados de música

In [ ]:

#segundo dataset


TEST_PROPORTION = 0.2
CICLES = 10
ETA = 0.5
MOMENTUM = 0.5

df = pd.read_csv('/content/default_features_1059_tracks.txt', header=None)

class_columns = pd.DataFrame(0, range(df.shape[0]), columns=[70, 71])

class_columns.iloc[: , 0] = (df.iloc[: , 68] + 90.0) / 180.0
class_columns.iloc[: , 1] = (df.iloc[: , 69] + 180.0) / 360.0
df = pd.concat([df,class_columns], axis=1)

def music_accuracy(test_proportion, cicles, eta, momentum, hidden_layers_num):
  TEST_PROPORTION = test_proportion
  CICLES = cicles
  ETA = eta
  MOMENTUM = momentum

  net_struct = []
  if hidden_layers_num == 2:
    net_struct = [68, 30, 30, 2]
  else:
    net_struct = [68, 30, 2]
  net = Net(net_struct)

  # embaralha as instâncias 
  shuffled = df.sample(frac=1)

  #numero de instancias do teste
  test_size = int(round(df.shape[0]*TEST_PROPORTION))

  #particiona em grupo de teste e grupo de treino
  teste = shuffled.iloc[0: test_size]
  treino = shuffled.iloc[test_size:]

  #TREINO

  #inputs contem os 68 tracks de musicas
  inputs = treino.drop(df.columns[[68,69,70,71]], axis=1).to_numpy()
  #targets contem a longitude e latitude da origem da musica
  targets = treino.iloc[:, [68,69,70,71]].to_numpy()

  inputs = normalize_inputs(inputs)
  net.online_train(inputs, targets[:, -2:], CICLES)

  #TESTE

  inputs = teste.drop(df.columns[[68,69,70,71]], axis=1).to_numpy()
  targets = teste.iloc[:, [68,69,70,71]].to_numpy()
  inputs = normalize_inputs(inputs)

  #modificar esse teste de acurácia 
  mean_error = 0

  outputs = np.empty( targets[:, :-2].shape )

  for idx, input_line in enumerate(inputs):
    res = net.forward(input_line.reshape(-1, 1))
    res = res.flatten()
    res[0] = res[0]*180.0 - 90.0
    res[1] = res[1]*360.0 - 180.0
    outputs[idx] = res

  mean_error = np.sqrt(((outputs - targets[:, :-2]) ** 2).mean())  
  return mean_error

In [ ]:
hidden_layer_num = [1,2]
num_cicles = [10,20,50,100]
eta_arr = [0.1,0.3,0.5,0.7]
momentum_arr = [0.3,0.4,0.5,0.6]
test_prop = [0.2,0.4,0.6,0.8]

#testes de acurácia:

#hidden_layers:
print("Número de Camadas: ")
for i in range(len(hidden_layer_num)):
  aux = []
  for j in range(5):
    aux.append(music_accuracy(0.2, 20, 0.1, 0.1, hidden_layer_num[i]))
  print(f"Valor {hidden_layer_num[i]} : {np.mean(aux)}")

#num_cicles:
print("Número de Ciclos: ")
for i in range(len(num_cicles)):
  aux = []
  for j in range(5):
    aux.append(music_accuracy(0.2, num_cicles[i], 0.1, 0.1, 1))
  print(f"Valor {num_cicles[i]} : {np.mean(aux)}")

#eta_arr:
print("Velocidade de aprendizado: ")
for i in range(len(eta_arr)):
  aux = []
  for j in range(5):
    aux.append(music_accuracy(0.2, 20, eta_arr[i], 0.1, 1))
  print(f"Valor {eta_arr[i]} : {np.mean(aux)}")

#momentum_arr:
print("Variação do Momentum: ")
for i in range(len(momentum_arr)):
  aux = []
  for j in range(5):
    aux.append(music_accuracy(0.2, 20, 0.1, momentum_arr[i], 1))
  print(f"Valor {momentum_arr[i]} : {np.mean(aux)}")

#test_prop:
print("Proporção de teste: ")
for i in range(len(test_prop)):
  aux = []
  for j in range(5):
    aux.append(music_accuracy(test_prop[i], 20, 0.1, 0.1, 1))
  print(f"Valor {test_prop[i]} : {np.mean(aux)}")

Número de Camadas: 
Valor 1 : 44.831108944215
Valor 2 : 37.63251584244816
Número de Ciclos: 
Valor 10 : 42.31655672153049
Valor 20 : 46.24289247078751
Valor 50 : 50.59857974078491
Valor 100 : 51.89033455604108
Velocidade de aprendizado: 
Valor 0.1 : 44.48724622296768
Valor 0.3 : 44.43770676948101
Valor 0.5 : 47.41494583526186
Valor 0.7 : 45.028901020563715
Variação do Momentum: 
Valor 0.3 : 45.76307720342224
Valor 0.4 : 45.12505346708609
Valor 0.5 : 44.53086451587617
Valor 0.6 : 45.10844184891774
Proporção de teste: 
Valor 0.2 : 44.967010965294484
Valor 0.4 : 46.5268284446918
Valor 0.6 : 47.70891327934302
Valor 0.8 : 47.00280550942885
